# Mini Fashion MNIST Coding Project
Build your own CNN on Fashion MNIST following the hints.

In [ ]:
# --- 1. Imports & Dataset ---
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist

In [ ]:
# --- 2. Load and preprocess dataset ---
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train = X_train / 255.0
X_test = X_test / 255.0
X_train = X_train[..., None]
X_test = X_test[..., None]

print('Training samples:', X_train.shape[0])
print('Test samples:', X_test.shape[0])

In [ ]:
# --- 3. Build a Simple CNN ---
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [ ]:
# --- 4. Compile the Model ---
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# --- 5. Train the Model ---
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))

In [ ]:
# --- 6. Evaluate the Model ---
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc:.4f}')

In [ ]:
# --- 7. Plot Training History ---
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training History')
plt.show()

In [ ]:
# --- 8. Make a Prediction Example ---
idx = np.random.randint(len(X_test))
plt.imshow(X_test[idx].reshape(28,28), cmap='gray')
plt.title(f'True label: {y_test[idx]}')
plt.show()
pred = model.predict(X_test[idx:idx+1])
print('Predicted class:', np.argmax(pred))